In [4]:
import numpy as np
from keras.utils import np_utils
import sys

In [5]:
file = open('full_speech.txt', encoding='utf8')
raw_text = file.read()

In [6]:
raw_text = raw_text.lower()

In [7]:
chars = sorted(list(set(raw_text)))
print(chars)

['\n', ' ', '!', '"', '$', '%', '&', "'", '(', ')', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', '[', ']', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\x85', '\x96', '\x97', '\xa0', 'é', '—']


In [8]:
bad_chars = ['(', ')','`','—']
for i in range(len(bad_chars)):
    raw_text=raw_text.replace(bad_chars[i],"")
chars = sorted(list(set(raw_text)))
print(chars)

['\n', ' ', '!', '"', '$', '%', '&', "'", ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', '[', ']', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\x85', '\x96', '\x97', '\xa0', 'é']


In [9]:
text_length = len(raw_text)
char_length = len(chars)
VOCABULARY = char_length
print("Text Length = "+ str(text_length))
print("No. of Characters = "+ str(char_length))

Text Length = 815819
No. of Characters = 58


In [10]:
SEQ_LENGTH = 100
char_to_int = dict((c, i)for i, c in enumerate(chars))
data_X = []
data_Y = []

for i in range(len(raw_text) - SEQ_LENGTH):
    X_text = raw_text[i: i+SEQ_LENGTH]
    X = [char_to_int[char] for char in X_text]
    data_X.append(X)
    Y = raw_text[i + SEQ_LENGTH]
    data_Y.append(char_to_int[Y])

In [11]:
length = len(data_X)
data_X = np.array(data_X)
data_X = np.reshape(data_X, (data_X.shape[0], data_X.shape[1], 1))
data_X = data_X/float(VOCABULARY)

data_Y = np.array(data_Y)
data_Y = np_utils.to_categorical(data_Y)
print(data_X.shape)
print(data_Y.shape)

(815719, 100, 1)
(815719, 58)


In [12]:
import keras 
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout, Activation
from tensorflow.keras.optimizers import RMSprop, Adam
from keras.callbacks import ModelCheckpoint

In [13]:
model = Sequential()

In [14]:
model.add(LSTM (256, input_shape= (SEQ_LENGTH, 1), return_sequences = True))
model.add(Dropout(0.2))
model.add(LSTM(256))
model.add(Dropout(0.2))

In [15]:
model.add(Dense (VOCABULARY, activation ='softmax'))

In [16]:
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam')

In [17]:
filepath="text_generation.h5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]
history = model.fit(data_X, data_Y, epochs = 40, batch_size = 128, callbacks = callbacks_list)

Epoch 1/40
6373/6373 [==============================] - 428s 64ms/step - loss: 2.7582

Epoch 00001: loss improved from inf to 2.53740, saving model to text_generation.h5
Epoch 2/40
6373/6373 [==============================] - 409s 64ms/step - loss: 2.1168

Epoch 00002: loss improved from 2.53740 to 2.04961, saving model to text_generation.h5
Epoch 3/40
6373/6373 [==============================] - 409s 64ms/step - loss: 1.88800s - loss: 1.8

Epoch 00003: loss improved from 2.04961 to 1.85771, saving model to text_generation.h5
Epoch 4/40
6373/6373 [==============================] - 410s 64ms/step - loss: 1.77224 - ETA - ETA: 0s - loss: 

Epoch 00004: loss improved from 1.85771 to 1.75397, saving model to text_generation.h5
Epoch 5/40
6373/6373 [==============================] - 412s 65ms/step - loss: 1.6981

Epoch 00005: loss improved from 1.75397 to 1.68671, saving model to text_generation.h5
Epoch 6/40
6373/6373 [==============================] - 409s 64ms/step - loss: 1.6424

Epoch 0

In [18]:
filename = 'text_generation.h5'
model.load_weights(filename)
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam')

In [19]:
initial_text = 'they get five killer terrorists that everybody wanted over there.we gets bergdahl.we get aaa traitor'
initial_text = [char_to_int[c] for c in initial_text]

In [20]:
int_to_char = dict((i, c) for i, c in enumerate(chars))
GENERATED_LENGTH = 100
test_text = initial_text
for i in range(len(initial_text)):
    X = np.reshape(test_text, (1, SEQ_LENGTH, 1))
    X  = X  / float(VOCABULARY)
    Prediction = model.predict(X)
    index = np.argmax(Prediction)
    result = int_to_char[index]
    sys.stdout.write(result)
    test_text.append(index)
    test_text = test_text[1:]
print ("\nDone.")

tienc in the world trade organization  and i will fight for every child in this country who want to
Done.
